---
title : "Belastingen inladen"
excecute:
    output: asis
---

Inlezen waterstanden uit verschillende bronsystemen:
- RWS Waterinfo
- FEWS
- (Noos) Matroos
- RWS WaterWebServices

Gebruikmakend van:
- csv bestanden voor uitwisselen van data

### Benodigde generieke Python packages

In [1]:
from pathlib import Path

### Definitie configuratie: CSV bestanden 

GlobalVariables:
    rootdir: "data_demo" 
    moments: [-24,0,24,48]  

    LoadsWaterinfo:
        parameters: ["waterhoogte"]
        MISSING_VALUE: 999

    LoadsFews:
        host: "https://fews.hhnk.nl"
        port: 443
        region: "fewspiservice"
        version: "1.25"
        filter: "HHNK_WEB"
        parameters: ["WATHTE [m][NAP][OW]", "WINDRTG [deg]"]
        aquo_equivalent: ["WATHTE","WINDRTG"]
        MISSING_VALUE: 999
    
    LoadsMatroos:
        website: "vitaal"
        model: "dcsm6_kf"
        parameters: ["WATHTE"]
        MISSING_VALUE: 999
    
    LoadsWaterwebservicesRWS:
        parameters: ["WATHTE"]
        MISSING_VALUE: 999

DataAdapter: 
    default_options:
        csv_source:
            sep: ";"
        csv:
            sep: ","
    BelastingLocaties_WaterInfo: 
        type: csv_source
        filter: "WaterInfo"
        path: "locaties.csv"
    Waterstanden_WaterInfo: 
        type: csv
        path: "waterstanden_waterinfo.csv"
    BelastingLocaties_FEWS: 
        type: csv_source
        filter: "FEWS"
        path: "locaties.csv"
    Waterstanden_FEWS: 
        type: csv
        path: "waterstanden_fews.csv"
    BelastingLocaties_Matroos: 
        type: csv_source
        filter: "NOOS Matroos"
        path: "locaties.csv"
    Waterstanden_Matroos: 
        type: csv
        path: "waterstanden_matroos.csv"
    BelastingLocaties_Waterwebservices_RWS: 
        type: csv_source
        filter: "RWS Waterwebservices"
        path: "locaties.csv"
    Waterstanden_Waterwebservices_RWS: 
        type: csv
        path: "waterstanden_waterwebservices_rws.csv"

### Lees configuratie in en maak dictionary base.Config aan

In [2]:
from toolbox_continu_inzicht import Config

yaml_config_file = "demo_sprint2-I.yaml"
path = Path.cwd() / "data_demo" / yaml_config_file
c = Config(config_path=path)
c.lees_config()

### Geef config mee aan base.Adapter

In [3]:
from toolbox_continu_inzicht import DataAdapter

data = DataAdapter(config=c)
print(f"Folder met data: {data.config.config_path}")
print(f"Globale variabelen: {data.config.global_variables}")
for data_adapter in data.config.data_adapters:
    print(f"Data apdater: {data_adapter}")

Folder met data: C:\Data\Python\TBCI\toolbox-continu-inzicht\docs\examples\notebooks\data_demo\demo_sprint2-I.yaml
Globale variabelen: {'rootdir': 'data_demo', 'moments': [-24, 0, 24, 48], 'LoadsWaterinfo': {'parameters': ['waterhoogte'], 'MISSING_VALUE': 999}, 'LoadsFews': {'host': 'https://fews.hhnk.nl', 'port': 443, 'region': 'fewspiservice', 'version': '1.25', 'filter': 'HHNK_WEB', 'parameters': ['WATHTE [m][NAP][OW]'], 'MISSING_VALUE': 999}, 'LoadsMatroos': {'website': 'noos', 'model': 'dcsm6_kf', 'parameters': ['WATHTE'], 'MISSING_VALUE': 999}, 'LoadsWaterwebservicesRWS': {'parameters': ['WATHTE'], 'MISSING_VALUE': 999}, 'calc_time': datetime.datetime(2025, 1, 20, 7, 0, tzinfo=datetime.timezone.utc)}
Data apdater: default_options
Data apdater: BelastingLocaties_WaterInfo
Data apdater: Waterstanden_WaterInfo
Data apdater: BelastingLocaties_FEWS
Data apdater: Waterstanden_FEWS
Data apdater: BelastingLocaties_Matroos
Data apdater: Waterstanden_Matroos
Data apdater: BelastingLocaties

### Invoerbestand (CSV) met data van belastinglocaties
Verplichte velden (schema):
- measurement_location_id: int64
- measurement_location_code: object (string)
- measurement_location_description: object (string)

measurement_location_id;measurement_location_code;measurement_location_description;source;tide\
1;Pannerdense-kop(PANN);Pannerdense kop;WaterInfo;false\
2;MPN-N-24;Meetpunt Spijkerboor;FEWS;false\
3;hoekvanholland;Hoek van Holland;NOOS Matroos;true\
4;9889;Hintham beneden;RWS Waterwebservices;false

### Laad module (functie) LoadsWaterinfo voor inlezen belastingen uit RWS Waterinfo

In [4]:
from toolbox_continu_inzicht.loads import LoadsWaterinfo

waterinfo = LoadsWaterinfo(data_adapter=data)

### Run module (functie) LoadsWaterinfo voor inlezen waterstanden uit Waterinfo
LET OP: zoek op basis van een measurement_location_code die in RWS WaterInfo overeenkomt met ['locationCodes'](https://waterinfo.rws.nl/api/chart/get?mapType=waterhoogte&locationCodes=Pannerdense-kop(PANN)&values=-48%2C48)

In [5]:
ds = waterinfo.run(input="BelastingLocaties_WaterInfo", output="Waterstanden_WaterInfo")

In [6]:
measuringlocations = waterinfo.df_in
measuringlocations

,measurement_location_id,measurement_location_code,measurement_location_description,source,tide
0,1,Pannerdense-kop(PANN),Pannerdense kop,WaterInfo,False


In [7]:
waterlevels = waterinfo.df_out
waterlevels

,measurement_location_id,measurement_location_code,measurement_location_description,parameter_id,parameter_code,parameter_description,unit,date_time,value,value_type
146,1,Pannerdense-kop(PANN),Pannerdense kop,4724,WATHTE,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,cm,2025-01-19 07:00:00+00:00,1010.0,meting
147,1,Pannerdense-kop(PANN),Pannerdense kop,4724,WATHTE,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,cm,2025-01-19 07:10:00+00:00,1009.0,meting
148,1,Pannerdense-kop(PANN),Pannerdense kop,4724,WATHTE,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,cm,2025-01-19 07:20:00+00:00,1009.0,meting
149,1,Pannerdense-kop(PANN),Pannerdense kop,4724,WATHTE,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,cm,2025-01-19 07:30:00+00:00,1009.0,meting
150,1,Pannerdense-kop(PANN),Pannerdense kop,4724,WATHTE,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,cm,2025-01-19 07:40:00+00:00,1009.0,meting
...,...,...,...,...,...,...,...,...,...,...
571,1,Pannerdense-kop(PANN),Pannerdense kop,4724,WATHTE,Waterhoogte verwacht Oppervlaktewater t.o.v. N...,cm,2025-01-22 04:20:00+00:00,946.0,verwachting
572,1,Pannerdense-kop(PANN),Pannerdense kop,4724,WATHTE,Waterhoogte verwacht Oppervlaktewater t.o.v. N...,cm,2025-01-22 04:30:00+00:00,945.0,verwachting
573,1,Pannerdense-kop(PANN),Pannerdense kop,4724,WATHTE,Waterhoogte verwacht Oppervlaktewater t.o.v. N...,cm,2025-01-22 04:40:00+00:00,945.0,verwachting
574,1,Pannerdense-kop(PANN),Pannerdense kop,4724,WATHTE,Waterhoogte verwacht Oppervlaktewater t.o.v. N...,cm,2025-01-22 04:50:00+00:00,945.0,verwachting


### Laad en run module (functie) LoadsFEWS voor inlezen belastingen uit FEWS
LET OP: zoek op basis van een measurement_location_code die in FEWS overeenkomt met ['locationId'](https://fews.hhnk.nl/FewsWebServices/rest/fewspiservice/v1/locations)<br>
![image info](data_demo/images/FEWS_locationIs.PNG)

Fews is niet extern bereikbaar

In [8]:
# from toolbox_continu_inzicht.loads import LoadsFews

# fews = LoadsFews(data_adapter=data)
# ds = fews.run(input="BelastingLocaties_FEWS", output="Waterstanden_FEWS")

In [9]:
# measuringlocations = fews.df_in
# measuringlocations

In [10]:
# waterlevels = fews.df_out
# waterlevels

### Laad en run module (functie) LoadsMatroos voor inlezen belastingen uit Matroos
LET OP: zoek op basis van een measurement_location_code die in (NOOS) Matroos overeenkomt met ['Location' -> loc_id!](https://vitaal.matroos.rws.nl/matroos/php/image_series.php?type=image&source_id=156&loc_id=7&unit_id=1&colors=blue&ymin=0&ymax=0&label=0&alarm=0&alert=0&prealert=0&localtime_offset=0&tcurrent=2024-11-11%2019:38:11&tstart=2024-11-10%2000:00:00&tstop=2024-11-13%2000:00:00&anal_time=2024-11-11%2000:00:00)

In [11]:
from toolbox_continu_inzicht.loads import LoadsMatroos

matroos = LoadsMatroos(data_adapter=data)
ds = matroos.run(input="BelastingLocaties_Matroos", output="Waterstanden_Matroos")

In [12]:
measuringlocations = matroos.df_in
measuringlocations

,measurement_location_id,measurement_location_code,measurement_location_description,source,tide
2,3,Hoek van Holland,Hoek van Holland,NOOS Matroos,True


In [13]:
waterlevels = matroos.df_out
waterlevels

,measurement_location_id,measurement_location_code,measurement_location_description,parameter_id,parameter_code,date_time,unit,value,value_type
0,3,Hoek van Holland,Hoek van Holland,4724,WATHTE,2025-01-19 07:00:00+00:00,cm,72.25,meting
1,3,Hoek van Holland,Hoek van Holland,4724,WATHTE,2025-01-19 07:10:00+00:00,cm,65.61,meting
2,3,Hoek van Holland,Hoek van Holland,4724,WATHTE,2025-01-19 07:20:00+00:00,cm,58.40,meting
3,3,Hoek van Holland,Hoek van Holland,4724,WATHTE,2025-01-19 07:30:00+00:00,cm,49.98,meting
4,3,Hoek van Holland,Hoek van Holland,4724,WATHTE,2025-01-19 07:40:00+00:00,cm,41.54,meting
...,...,...,...,...,...,...,...,...,...
404,3,Hoek van Holland,Hoek van Holland,4724,WATHTE,2025-01-22 02:20:00+00:00,cm,-43.96,verwachting
405,3,Hoek van Holland,Hoek van Holland,4724,WATHTE,2025-01-22 02:30:00+00:00,cm,-42.83,verwachting
406,3,Hoek van Holland,Hoek van Holland,4724,WATHTE,2025-01-22 02:40:00+00:00,cm,-41.31,verwachting
407,3,Hoek van Holland,Hoek van Holland,4724,WATHTE,2025-01-22 02:50:00+00:00,cm,-39.64,verwachting


### Laad en run module (functie) LoadsWaterwebservicesRWS voor inlezen belastingen uit RWS Waterwebservices
LET OP: zoek op basis van een measurement_location_code die in RWS WaterWebservices overeenkomt met Locatie_MessageID!\
![image info](data_demo/images/WaterWebservicesRWS_Locatie_MessageId.PNG)

In [14]:
from toolbox_continu_inzicht.loads import LoadsWaterwebservicesRWS

waterwebservices_rws = LoadsWaterwebservicesRWS(data_adapter=data)
ds = waterwebservices_rws.run(
    input="BelastingLocaties_Waterwebservices_RWS",
    output="Waterstanden_Waterwebservices_RWS",
)

In [15]:
measuringlocations = waterwebservices_rws.df_in
measuringlocations

,measurement_location_id,measurement_location_code,measurement_location_description,source,tide
3,4,9889,Hintham beneden,RWS Waterwebservices,False


In [16]:
waterlevels = waterwebservices_rws.df_out
waterlevels

,measurement_location_id,measurement_location_code,measurement_location_description,parameter_id,parameter_code,date_time,unit,value,value_type
0,4,9889,Hintham beneden,4724,WATHTE,2025-01-19 08:00:00+01:00,cm,197.0,meting
1,4,9889,Hintham beneden,4724,WATHTE,2025-01-19 08:10:00+01:00,cm,197.0,meting
2,4,9889,Hintham beneden,4724,WATHTE,2025-01-19 08:20:00+01:00,cm,197.0,meting
3,4,9889,Hintham beneden,4724,WATHTE,2025-01-19 08:30:00+01:00,cm,197.0,meting
4,4,9889,Hintham beneden,4724,WATHTE,2025-01-19 08:40:00+01:00,cm,197.0,meting
...,...,...,...,...,...,...,...,...,...
142,4,9889,Hintham beneden,4724,WATHTE,2025-01-20 07:40:00+01:00,cm,196.0,meting
143,4,9889,Hintham beneden,4724,WATHTE,2025-01-20 07:50:00+01:00,cm,197.0,meting
144,4,9889,Hintham beneden,4724,WATHTE,2025-01-20 08:00:00+01:00,cm,195.0,meting
145,4,9889,Hintham beneden,4724,WATHTE,2025-01-20 08:10:00+01:00,cm,198.0,verwachting


### Grafische weergave van resultaten in 4 grafieken